1. download feature table of whatever complete organism
2. parse name column for search term of interest (e.g. nitrile hydratase + related terms)
	if no, download genome (fasta) to neg directory and increase a neg_counter
	(also download gbk file and parse organism and isolation source out of it)
		if counter is hit, stop downloading negative genomes
	if yes, download genome (fasta) to pos directory and increase a pos_counter
	(also download gbk file and parse organism and isolation source out of it - could also parse the sequence for TP genes out of it and assemble it automatically)
		if counter is hit, stop downloading pos genomes
	stop downloading once both counters are hit
3. pass directories on to the make coverage table scripts


Also I need a protein alignment of homologues, so should possibly download more positive genomes than I actually need to be able to build the 
alignment as well.

IMPORTANT! How to validate TPs?


YcaO: Currently downloaded 49 of 50 pos genome(s).
Currently downloaded 150 of 150 neg genome(s).
Checked a total of:	 1019 genome(s). 

In [18]:
#Output directory will be named after BGC_type
BGC_type = 'Rieske'
#amount genomes does not equal amount sequences as 1 genome can have multiple hits for a search term.
amount_neg_genomes = 25
amount_pos_genomes = 15 #use some as TP genes and make prot alignment from some others
#names are converted to lower case for searching, so search term should be all lower case letters
search_term = 'lanthionine synthetasecC family protein'
#if isolation source doesn't matter, include '' and 'not annotated' in the filter
#isolation source information is also converted to lower case, so keep search terms lower case
#can modify positive and negative isolation source separately.
neg_isolation_source_filter = ['', 'not annotated', 'marine', 'sea', 'sponge', 'ocean', 'porifera', 'seafloor', 'sediment', 'water', 'tidal', 'coral', 'reef', 'coast', 'ship', 'fish', 'aquaculture', 'atlantic', 'pacific', 'mediterranean', 'baltic', 'pond']
pos_isolation_source_filter =  ['', 'not annotated', 'marine', 'sea', 'sponge', 'ocean', 'porifera', 'seafloor', 'sediment', 'water', 'tidal', 'coral', 'reef', 'coast', 'ship', 'fish', 'aquaculture', 'atlantic', 'pacific', 'mediterranean', 'baltic', 'pond']

In [19]:
import os
from os import listdir, mkdir
from os.path import isfile, join
from pathlib import Path
import pandas as pd
import random
import glob
from Bio import SeqIO

In [20]:
def makedir(dirpath):
    if os.path.isdir(dirpath):
        print(dirpath,'exists already')
    else:
        print('Making', dirpath)    
        os.mkdir(dirpath)

        
# Defining paths for required directory structure for input and output files relative to parent directory
parent_dir='/nesi/project/vuw03285/'
BGC_path=os.path.join(parent_dir, BGC_type)
base_genomes_path=os.path.join(BGC_path, 'base_genomes')
neg_genomes_path=os.path.join(BGC_path, 'base_genomes/temp_neg_genomes')
pos_genomes_path=os.path.join(BGC_path, 'base_genomes/temp_pos_genomes')
feature_tables_path=os.path.join(BGC_path, 'base_genomes/feature_tables')
neg_gbk_path=os.path.join(BGC_path, 'base_genomes/neg_gbk_files')
pos_gbk_path=os.path.join(BGC_path, 'base_genomes/pos_gbk_files')



# Calling function to make directories if they don't exist yet
makedir(BGC_path)
makedir(base_genomes_path)
makedir(neg_genomes_path)
makedir(pos_genomes_path)
makedir(feature_tables_path)
makedir(neg_gbk_path)
makedir(pos_gbk_path)

os.chdir(parent_dir)

Making /nesi/project/vuw03285/Rieske
Making /nesi/project/vuw03285/Rieske/base_genomes
Making /nesi/project/vuw03285/Rieske/base_genomes/temp_neg_genomes
Making /nesi/project/vuw03285/Rieske/base_genomes/temp_pos_genomes
Making /nesi/project/vuw03285/Rieske/base_genomes/feature_tables
Making /nesi/project/vuw03285/Rieske/base_genomes/neg_gbk_files
Making /nesi/project/vuw03285/Rieske/base_genomes/pos_gbk_files


In [21]:
%%capture cap --no-stderr
# Generating a report file for this particular script
print('\nBGC_type =', BGC_type)
print('\namount_neg_genomes =', amount_neg_genomes)
print('\namount_pos_genomes =', amount_pos_genomes)
print('\nsearch_term =', search_term)
print('\nneg_isolation_source_filter =', neg_isolation_source_filter)
print('\npos_isolation_source_filter =', pos_isolation_source_filter, '\n')
with open(BGC_path+'/'+'report_download_base_genomes.txt', 'w') as f:
    f.write(str(cap))

Using ncbi-genome-download to get a list of accession numbers of complete bacterial and archaeal genomes from refseq. Save this file as a tsv.
Run only if file doesn't exist yet

In [8]:
#%%bash
# Download table of GCF accessions
#including archaea: 21739 (374 more)
#excluding archaea: 21365
#/home/blankma/.local/bin/ncbi-genome-download bacteria,archaea -s refseq -F features -l complete --flat-output --dry-run > /nesi/project/vuw03285/complete_refseq_accessions.tsv

In [4]:
# read .tsv file into a pandas dataframe
gcf_df = pd.read_csv('complete_refseq_accessions.tsv', header=None, sep='\t', skiprows=[0])
gcf_df.columns=['accession','organism','strain'] 

#turn first column into a list of accession number to randomly select from
accession_list = list(gcf_df.loc[:,'accession'])

In [9]:
# https://stackoverflow.com/questions/54662605/how-to-pass-a-python-variables-to-shell-script-in-azure-databricks-notebookbles
#os.environ['LIST'] = ' '.join(random_gcf)
#print(os.getenv('LIST'))

- Also make a shutoff condition if coming back to download more genomes. I.e. parse feature table folder and see if GCF_... already exists, if so, skip (also should append to summary file in this case, but that's probably too much)
- pos only appended to summary if the protein id is in the last or only feature

In [5]:
####TINKER CELL####
# This cell is a modified version of the previous one with the difference that
# the isolation source can be filtered for, here.


#scramble accession_list
random_gcf_list=random.sample(accession_list,len(accession_list))

# pre-definiting headers and data types for faster loading of data frames
feature_table_headers = ["# feature", "class", "assembly", "assembly_unit", "seq_type", "chromosome", "genomic_accession", "start", "end", "strand", "product_accession", "non-redundant_refseq", "related_accession", "name", "symbol", "GeneID", "locus_tag", "feature_interval_length", "product_length", "attributes"]
data_types = {'# feature':str,'class':str,'assembly':str,'assembly_unit':str,'seq_type':str,'chromosome':str,'genomic_accession':str,'strand':str,'product_accession':str,'non-redundant_refseq':str,'related_accession':str,'name':str,'symbol':str,'locus_tag':str,'attributes':str}

#initiate summary dictionary
summary_dict={'dir':[],'assembly':[],'isolation_source':[],'organism':[],'description':[],'protein_id':[],'sequence':[]}

#initialise counters to keep track of the amount of checked and downloaded genomes
gcf_checked_counter=0
neg_genomes_counter=0
pos_genomes_counter=0


#open a file to parse positive sequences into --> generate the TP file for this gene

for i in range(0,len(random_gcf_list)): 

    os.chdir(feature_tables_path)
    print('\nCurrently downloaded',pos_genomes_counter, 'of', amount_pos_genomes, 'pos genome(s).')
    print('Currently downloaded',neg_genomes_counter, 'of', amount_neg_genomes, 'neg genome(s).')


    #exit from the loop once the pre-set amount of genomes has been downloaded
    if neg_genomes_counter == amount_neg_genomes and pos_genomes_counter == amount_pos_genomes:
        print('\n\n\nDownloaded specified amount of genomes.')
        break
       
    print('Checked a total of:\t', gcf_checked_counter, 'genome(s). \n\n\n')
    gcf_checked_counter+=1
    
    #exit from the loop if all genomes have been checked (~22000, so that would take a while)
    if gcf_checked_counter == len(random_gcf_list):
        print('Checked all possible genomes. Exiting.')
        break

        
    random_gcf=random_gcf_list[i]
    !/home/blankma/.local/bin/ncbi-genome-download bacteria,archaea -s refseq -F features -l complete -A "{random_gcf}" --flat-output --output-folder "{feature_tables_path}"
    !gunzip -f "{feature_tables_path}"/"{random_gcf}"*
        
    # Pre-filter individual feature table by a range of criteria to make it more compact
    for feature_table in glob.glob(random_gcf+'*_feature_table.txt'):
        print('Filtering', feature_table)
        features_df = pd.read_csv(feature_table, index_col=None, sep = '\t', names = feature_table_headers, dtype=data_types)
        mask = (features_df['# feature'] == 'CDS')
        export_df = features_df[mask]
        mask2 = (export_df['class'] == 'with_protein') # removes pseudogenes 'without protein'
        export_df2 = export_df[mask2]
        mask3 = (export_df2['assembly_unit'] == 'Primary Assembly')
        export_df3 = export_df2[mask3]
        mask4 = (export_df3['seq_type'] == 'chromosome') | (export_df3['seq_type'] == 'plasmid')
        export_df4 = export_df3[mask4]
        mask5 = pd.isnull(export_df4['attributes'])
        export_df5 = export_df4[mask5]
        export_df5.loc[:,'name'] = export_df5.loc[:,'name'].str.lower()
        mask6 = (~export_df5['name'].str.contains('hypothetical protein'))
        export_df6 = export_df5[mask6]

        
        # search for search term in reduced features table and then conditionally download fasta and gbk either into
        # pos or neg genomes directories
        interest_mask = export_df6['name'].str.contains(search_term)
        search_names = export_df6[interest_mask]    
        

        
##### NEGATIVE GENOMES #####

    # if search term is not found in genome and the set amount of negative genomes has not been reached
    if len(search_names) == 0 and neg_genomes_counter < amount_neg_genomes:
        
        # Download gbk file
        print('No hits for', search_term, 'in', random_gcf)
        print('Downloading gbk of', random_gcf, 'to negative gbk directory')
        !/home/blankma/.local/bin/ncbi-genome-download bacteria,archaea -s refseq -F genbank -l complete -A "{random_gcf}" --flat-output --output-folder "{neg_gbk_path}"
        !gunzip -f "{neg_gbk_path}"/"{random_gcf}"*
 
        #parse gbk files for isolation source and organism
        os.chdir(neg_gbk_path)
        for gbk_file in glob.glob(random_gcf+'*.gbff'):
            print('Parsing gbk file of', random_gcf, 'for organism and isolation source.')
            with open(gbk_file):
                for record in SeqIO.parse(gbk_file,'gb'):
                    features = record.features[0]
                    
                    #parse organism if annotated, handle if not
                    if 'organism' in features.qualifiers:
                        organism = features.qualifiers['organism']
                    else:
                        organism = ['organism not annotated']

                    #parse isolation source if annotated, handle if not
                    if 'isolation_source' in features.qualifiers:
                        isolation_source = [entry.lower() for entry in record.features[0].qualifiers['isolation_source']]
                    else:
                        isolation_source = ['isolation_source not annotated']

                # if predefined search words are found in isolation source, increment a counter
                #download_indicator = 0
                wordlist = []
                for word in neg_isolation_source_filter:
                    if word in isolation_source[0]:
                        #download_indicator+=1
                        wordlist.append(word)
                print(wordlist, 'in desired isolation source list.')

                        
                # if isolation source condition is met...
                if len(wordlist) >= 1:
                    #...add row with required information to the summary table...
                    print('Add row for', random_gcf, 'in', record.description, 'to summary table')
                    summary_dict['dir'].append('-')
                    summary_dict['assembly'].append(random_gcf)
                    summary_dict['isolation_source'].append(isolation_source[0])
                    summary_dict['organism'].append(organism[0])
                    summary_dict['description'].append(record.description)
                    summary_dict['protein_id'].append('None')
                    summary_dict['sequence'].append('None')

                    #...and download the fasta file for the corresponding genome and increment the counter.
                    os.chdir(feature_tables_path)
                    print('Downloading fasta of', random_gcf, 'to negative genomes directory.')
                    !/home/blankma/.local/bin/ncbi-genome-download bacteria,archaea -s refseq -F fasta -l complete -A "{random_gcf}" --flat-output --output-folder "{neg_genomes_path}"
                    !gunzip -f "{neg_genomes_path}"/"{random_gcf}"*
                    #download_indicator = 0
                    wordlist = []
                    neg_genomes_counter+=1

                # if isolation source condition is not met...
                else:
                    print(random_gcf, 'negative for', search_term, 'but', isolation_source[0], 'not in desired list.')
                    print('Removing gbk file and feature table of', random_gcf)
                    !rm "{neg_gbk_path}"/"{random_gcf}"*
                    !rm "{feature_tables_path}"/"{random_gcf}"*
                    wordlist = []
                        

    # if search term is not found in genome and the set amount of negative genomes has been reached
    elif len(search_names) == 0 and neg_genomes_counter == amount_neg_genomes:
        print(random_gcf, 'is negative for', search_term+',', 'but specified number of negative genomes reached.')
        print('Removing feature table of', random_gcf)
        !rm "{feature_tables_path}"/"{random_gcf}"*
    
    
##### POSITIVE GENOMES #####
    # if search term is found in genome and the set amount of positive genomes has not been reached
    elif len(search_names) != 0 and pos_genomes_counter < amount_pos_genomes:

        # Download gbk file
        product_accessions = list(search_names.loc[:,'product_accession'])
        print('Found following hit/s for search term', search_term+':', product_accessions)
        print('Downloading gbk of', random_gcf, 'to positive gbk directory.')
        !/home/blankma/.local/bin/ncbi-genome-download bacteria,archaea -s refseq -F genbank -l complete -A "{random_gcf}" --flat-output --output-folder "{pos_gbk_path}"
        !gunzip -f "{pos_gbk_path}"/"{random_gcf}"*
        
        
        # Adding an entry for each found hit in each genome into summary_dict
        os.chdir(pos_gbk_path)
        for gbk_file in glob.glob(random_gcf+'*.gbff'):
            print('Parsing gbk file of', random_gcf, 'for organism and isolation source.')
            with open(gbk_file):
                for record in SeqIO.parse(gbk_file,'gb'):    
                    features = record.features[0]
                    
                    #parse organism if annotated, handle if not
                    if 'organism' in record.features[0].qualifiers:
                        organism = record.features[0].qualifiers['organism']
                    else:
                        organism = ['organism not annotated']

                    #parse isolation source if annotated, handle if not
                    if 'isolation_source' in record.features[0].qualifiers:
                        isolation_source = [entry.lower() for entry in record.features[0].qualifiers['isolation_source']]
                        record.features[0].qualifiers['isolation_source']
                    else:
                        isolation_source = ['isolation_source not annotated']
        
        
                    # if predefined search words are found in isolation source, increment a counter
                    #download_indicator = 0
                    wordlist = []
                    for word in pos_isolation_source_filter:
                        if word in isolation_source[0]:
                            #download_indicator+=1
                            wordlist.append(word)
                    

                    # if isolation source condition is met...
                    if len(wordlist) >= 1:
                        #...add row with required information to the summary table...  
                        for i in record.features:
                            for j in product_accessions:
                                if i.qualifiers.get('protein_id') == [j]:
                                    #print(i.extract(record.seq))
                                    print('Add row for', j, 'in', record.description, 'to summary table')
                                    summary_dict['dir'].append('+')
                                    summary_dict['assembly'].append(random_gcf)
                                    summary_dict['isolation_source'].append(isolation_source[0])
                                    summary_dict['organism'].append(organism[0])
                                    summary_dict['description'].append(record.description)
                                    summary_dict['protein_id'].append(j)
                                    summary_dict['sequence'].append(i.qualifiers.get('translation'))
                                    print('Adding nucleotide sequence of', j, 'to TP file.')
                                    with open(BGC_path+'/'+BGC_type+'_tp_genes.fasta', 'a') as fasta_file:
                                        fasta_file.write('>'+random_gcf+'_'+organism[0]+'_'+j+'\n'+str(i.extract(record.seq))+'\n')



                                
        #...and download the fasta file for the corresponding genome and increment the counter.
        if len(wordlist) >= 1:
            os.chdir(feature_tables_path)
            print(wordlist, 'in desired isolation source list.')
            print('Downloading fasta of', random_gcf, 'to positive genomes directory')
            !/home/blankma/.local/bin/ncbi-genome-download bacteria,archaea -s refseq -F fasta -l complete -A "{random_gcf}" --flat-output --output-folder "{pos_genomes_path}"
            !gunzip -f "{pos_genomes_path}"/"{random_gcf}"*
            pos_genomes_counter+=1
            wordlist = []
        else:
            print(random_gcf, 'positive for', search_term, 'but', isolation_source[0], 'not in desired list.')
            print('Removing gbk file and feature table of', random_gcf)
            !rm "{pos_gbk_path}"/"{random_gcf}"*
            !rm "{feature_tables_path}"/"{random_gcf}"*
            wordlist = []


    # if search term is found in genome and the set amount of positive genomes has been reached
    elif len(search_names) != 0 and pos_genomes_counter == amount_pos_genomes:
        print(random_gcf, 'is positive for', search_term+',', 'but specified number of positive genomes reached.')
        print('Removing feature table of', random_gcf)
        !rm "{feature_tables_path}"/"{random_gcf}"*

        
        
        
##### FAIL? #####
        
    elif neg_genomes_counter == amount_neg_genomes and pos_genomes_counter == amount_pos_genomes:
        print("This shouldn't be happening", '\n')   
        

print('Done')

              
              
              
summary_df = pd.DataFrame(summary_dict)
summary_df.to_csv(BGC_path+'/summary.tsv', index=False, sep='\t')


Currently downloaded 0 of 15 pos genome(s).
Currently downloaded 0 of 25 neg genome(s).
Checked a total of:	 0 genome(s). 



Filtering GCF_008330925.1_ASM833092v1_feature_table.txt
No hits for rieske 2fe-2s domain-containing protein in GCF_008330925.1


/home/blankma/.local/lib/python3.8/site-packages/pandas/core/indexing.py:1843: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item_labels[indexer[info_axis]]] = value


Parsing gbk file of GCF_008330925.1 for organism and isolation source.
['', 'not annotated', 'isolation_source not annotated'] in desired isolation source list.
Add row for GCF_008330925.1 in Neisseria meningitidis strain 13-600 chromosome, complete genome to summary table

Currently downloaded 0 of 15 pos genome(s).
Currently downloaded 1 of 25 neg genome(s).
Checked a total of:	 1 genome(s). 



Filtering GCF_003382725.1_ASM338272v1_feature_table.txt
No hits for rieske 2fe-2s domain-containing protein in GCF_003382725.1
Parsing gbk file of GCF_003382725.1 for organism and isolation source.
['', 'not annotated', 'isolation_source not annotated'] in desired isolation source list.
Add row for GCF_003382725.1 in Enterobacter hormaechei subsp. xiangfangensis strain OSUVMCKPC4-2 plasmid pOSUEC_F, complete sequence to summary table

Currently downloaded 0 of 15 pos genome(s).
Currently downloaded 2 of 25 neg genome(s).
Checked a total of:	 2 genome(s). 



Filtering GCF_001708555.1_ASM17085


Currently downloaded 4 of 15 pos genome(s).
Currently downloaded 8 of 25 neg genome(s).
Checked a total of:	 12 genome(s). 



Filtering GCF_001610935.1_ASM161093v1_feature_table.txt
No hits for rieske 2fe-2s domain-containing protein in GCF_001610935.1
Parsing gbk file of GCF_001610935.1 for organism and isolation source.
['', 'not annotated', 'isolation_source not annotated'] in desired isolation source list.
Add row for GCF_001610935.1 in Streptococcus suis strain 90-1330 chromosome, complete genome to summary table

Currently downloaded 4 of 15 pos genome(s).
Currently downloaded 9 of 25 neg genome(s).
Checked a total of:	 13 genome(s). 



Filtering GCF_005576555.1_ASM557655v1_feature_table.txt
No hits for rieske 2fe-2s domain-containing protein in GCF_005576555.1
Parsing gbk file of GCF_005576555.1 for organism and isolation source.
[''] in desired isolation source list.
Add row for GCF_005576555.1 in Collinsella aerofaciens strain P10wA7 chromosome, complete genome to summary t


Currently downloaded 7 of 15 pos genome(s).
Currently downloaded 14 of 25 neg genome(s).
Checked a total of:	 21 genome(s). 



Filtering GCF_005938665.2_ASM593866v2_feature_table.txt
Found following hit/s for search term rieske 2fe-2s domain-containing protein: ['WP_026286598.1', 'WP_138867373.1', 'WP_019648621.1', 'WP_138866314.1']
Parsing gbk file of GCF_005938665.2 for organism and isolation source.
Add row for WP_026286598.1 in Pseudomonas sp. ATCC 43928 chromosome, complete genome to summary table
Adding nucleotide sequence of WP_026286598.1 to TP file.
Add row for WP_138867373.1 in Pseudomonas sp. ATCC 43928 chromosome, complete genome to summary table
Adding nucleotide sequence of WP_138867373.1 to TP file.
Add row for WP_019648621.1 in Pseudomonas sp. ATCC 43928 chromosome, complete genome to summary table
Adding nucleotide sequence of WP_019648621.1 to TP file.
Add row for WP_138866314.1 in Pseudomonas sp. ATCC 43928 chromosome, complete genome to summary table
Adding nucleo


Currently downloaded 9 of 15 pos genome(s).
Currently downloaded 24 of 25 neg genome(s).
Checked a total of:	 33 genome(s). 



Filtering GCF_011404475.1_ASM1140447v1_feature_table.txt
No hits for rieske 2fe-2s domain-containing protein in GCF_011404475.1
Parsing gbk file of GCF_011404475.1 for organism and isolation source.
[''] in desired isolation source list.
Add row for GCF_011404475.1 in Escherichia coli strain 2018-06-4CC chromosome, complete genome to summary table

Currently downloaded 9 of 15 pos genome(s).
Currently downloaded 25 of 25 neg genome(s).
Checked a total of:	 34 genome(s). 



Filtering GCF_012935005.1_ASM1293500v1_feature_table.txt
Found following hit/s for search term rieske 2fe-2s domain-containing protein: ['WP_001277084.1']
Parsing gbk file of GCF_012935005.1 for organism and isolation source.
Add row for WP_001277084.1 in Acinetobacter baumannii strain VB2486 chromosome, complete genome to summary table
Adding nucleotide sequence of WP_001277084.1 to TP fi


Currently downloaded 13 of 15 pos genome(s).
Currently downloaded 25 of 25 neg genome(s).
Checked a total of:	 49 genome(s). 



Filtering GCF_001896025.1_ASM189602v1_feature_table.txt
GCF_001896025.1 is negative for rieske 2fe-2s domain-containing protein, but specified number of negative genomes reached.
Removing feature table of GCF_001896025.1

Currently downloaded 13 of 15 pos genome(s).
Currently downloaded 25 of 25 neg genome(s).
Checked a total of:	 50 genome(s). 



Filtering GCF_900638315.1_57195_D02_feature_table.txt
GCF_900638315.1 is negative for rieske 2fe-2s domain-containing protein, but specified number of negative genomes reached.
Removing feature table of GCF_900638315.1

Currently downloaded 13 of 15 pos genome(s).
Currently downloaded 25 of 25 neg genome(s).
Checked a total of:	 51 genome(s). 



Filtering GCF_900087795.2_WHOV_feature_table.txt
GCF_900087795.2 is negative for rieske 2fe-2s domain-containing protein, but specified number of negative genomes reached